## maze

maze as **two-dimensional-array** based of `Cell`-objects

In [30]:
from enum import Enum
from typing import List, NamedTuple, Callable, Optional
import random
from math import sqrt
from __future__ import annotations

T = TypeVar('T')

In [8]:
class Cell(str, Enum):
    EMPTY = " "
    BLOCKED = "X"
    START = "S"
    GOAL = "G"
    PATH = "*"

need to access singular position in maze

In [9]:
class MazeLocation(NamedTuple):
    row: int
    column: int

`Maze`Class

In [33]:
class Maze:
    def __init__(self, rows: int = 10, columns: int = 10, sparseness: float = 0.2, start: MazeLocation = MazeLocation(0,0), goal: MazeLocation = MazeLocation(9,9)) -> None:
        # initialise instancevariables
        self._rows: int = rows
        self._columns: int = columns
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal

        # filling grid with empty cells
        self._grid: List[List[Cell]] = [[Cell.EMPTY for c in range(columns)] for r in range(rows)]

        # filling grid with random cells
        self._randomly_fill(rows, columns, sparseness)

        # paste start and goal position
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL


    def _randomly_fill(self, rows: int, columns: int, sparseness: float):
        for row in range(rows):
            for column in range(columns):
                if random.uniform(0, 1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED

    def __str__(self) -> str:
        output: str = ""
        for row in self._grid:
            output += "".join([c.value for c in row]) + "\n"
        return output

    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal

    def successors(self, ml: MazeLocation) -> List[MazeLocation]:
        locations: List[MazeLocation] = []
        if ml.row + 1 < self._rows and self._grid[ml.row + 1][ml.column]!= Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ml.column + 1 < self._columns and self._grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations

    def mark(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.PATH
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL

    def clear(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.EMPTY
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL

In [37]:
maze: Maze = Maze()
print(maze)

S   XX X  
   X     X
X         
 X        
 X    X   
      X   
        XX
XX X    X 
   X      
   X X   G



### depth-first-search

based of a stack

In [28]:
class Stack(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []
    @property
    def empty(self) -> bool:
        return not self._container

    def push(self, item: T) -> None:
        self._container.append(item)

    def pop(self) -> T:
        return self._container.pop()  # LIFO

    def __repr__(self) -> str:
        return repr(self._container)

and a Node

In [31]:
class Node(Generic[T]):
    def __init__(self, state: T, parent: Optional[Node], cost: float = 0.0, heuristic: float = 0.0) -> None:
        self.state: T = state
        self.parent: Optional[Node] = parent
        self.cost: float = cost
        self.heuristic: float = heuristic

    def __lt__(self, other: Node) -> bool:
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

In [39]:
def dfs(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]]) -> Optional[Node[T]]:
    # frontier indicates unvisited places
    frontier: Stack[Node[T]] = Stack()
    frontier.push(Node(initial, None))
    # explored indicates already visited places
    explored: Set[T] = {initial}

    # continue, as long as there are unvisited places
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        # if goal-destination, then done
        if goal_test(current_state):
            return current_node
        # check where to go next and where we haven't visited yet
        for child in successors(current_state):
            if child in explored:  # skip already visited childs
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None  # searched everything but never found target

if `dfs()` is successfull, path can be reconstructed using `Node`-objects and their `.parent`-function

In [32]:
def node_to_path(node: Node[T]) -> List[T]:
    path: List[T] = [node.state]
    # backwards from end to start
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
    path.reverse()
    return path

## dfs-test

In [43]:
m: Maze = Maze()
print(m)
solution1: Optional[Node[MazeLocation]] = dfs(m.start, m.goal_test, m.successors)

if solution1 is None:
    print("no solution found via dfs")
else:
    path1: List[MazeLocation] = node_to_path(solution1)
    m.mark(path1)
    print(m)
    m.clear(path1)

S  XX  X  
    X    X
 X      X 
    X   X 
 XX       
       X  
    X   X 
  X    X  
    X  X  
        XG

S**XX  X  
  **X    X
 X *****X 
    X  *X 
 XX    ***
       X *
    X   X*
  X    X *
    X  X *
        XG

